## Relevant links

Model itself is based on this Kaggle Kernel: https://www.kaggle.com/hsinwenchang/keras-mobilenet-data-augmentation-visualize

iNaturalist data 2017 download: https://github.com/visipedia/inat_comp/blob/master/2017/README.md

iNaturalist data 2018 download: https://github.com/visipedia/inat_comp

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D,MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras import optimizers


Using TensorFlow backend.


In [2]:
import h5py

# tests recommended by: http://docs.h5py.org/en/stable/build.html
# package needed to convert Keras model -> TensorFlow Lite
h5py.run_tests()

.....................................................x.........................s.........................................x....................................s...s......ss.......................................................................................................ssssss...................................................................x....x.........................x......x.................................................ssss.....s............
----------------------------------------------------------------------
Ran 457 tests in 1.507s

OK (skipped=16, expected failures=6)


<unittest.runner.TextTestResult run=457 errors=0 failures=0>

In [12]:
os.getcwd()

'/Users/kelson/Kaggle'

In [13]:
# Read in unscrambled category data

# (TODO: Adjust this path based on where downloaded data + info is)
path_to_categories_json = '../Downloads/categories.json'
with open(path_to_categories_json) as data_file_categories:
    categories = json.load(data_file_categories)
    
# Read in Kingdom and category ID info
# categories_df = pd.DataFrame(categories)[['kingdom', 'id', 'name', 'genus']].rename(columns={'id':'category_id'})
categories_df = pd.DataFrame(categories).rename(columns={'id':'category_id'})

In [14]:
categories_df.head()

,class,family,genus,category_id,kingdom,name,order,phylum,supercategory
0,Polychaeta,Amphinomidae,Hermodice,0,Animalia,Hermodice carunculata,Phyllodocida,Annelida,Animalia
1,Polychaeta,Sabellariidae,Phragmatopoma,1,Animalia,Phragmatopoma californica,Sabellida,Annelida,Animalia
2,Polychaeta,Sabellidae,Eudistylia,2,Animalia,Eudistylia vancouveri,Sabellida,Annelida,Animalia
3,Polychaeta,Serpulidae,Galeolaria,3,Animalia,Galeolaria hystrix,Sabellida,Annelida,Animalia
4,Polychaeta,Serpulidae,Serpula,4,Animalia,Serpula columbiana,Sabellida,Annelida,Animalia


In [27]:
def retrieve_and_trim(path):
    # Load Data from JSON
    with open(path) as data_file:
        anns = json.load(data_file)
    # Read in Image + Category Pairs
    anns_df = pd.DataFrame(anns['annotations'])[['image_id', 'category_id']]
    
    # Read in Image + File_Name Pairs
    img_df = pd.DataFrame(anns['images'])[['id', 'file_name']].rename(columns={'id':'image_id'})

    # Merge 
    df_file_cat = pd.merge(img_df, anns_df, on='image_id')
    
    # Add category name to file
    df_file_w_kingdom = pd.merge(df_file_cat, categories_df, on='category_id')
    
    # converting to string to use sparse_categorical_crossentropy
    df_file_w_kingdom['category_id'] = df_file_w_kingdom['category_id'].astype(str)
    
    # Retain only the records with the category as Plantae or Fungi
    df_file_w_kingdom = df_file_w_kingdom[df_file_w_kingdom['kingdom'].isin(['Plantae', 'Fungi'])]

    return df_file_w_kingdom
    
    
    

In [28]:
path_to_train_data = '../Downloads/train2018.json'
path_to_val_data = '../Downloads/val2018.json'

# Retrieve and parse data
train_df = retrieve_and_trim(path_to_train_data)
val_df = retrieve_and_trim(path_to_val_data)

train_df.head()

,image_id,file_name,category_id,class,family,genus,kingdom,name,order,phylum,supercategory
0,1,train_val2018/Plantae/7477/3b60c9486db1d2ee875...,7477,Magnoliopsida,Platanaceae,Platanus,Plantae,Platanus occidentalis,Proteales,Magnoliophyta,Plantae
1,4336,train_val2018/Plantae/7477/3d5d7f1e0eedb1df3aa...,7477,Magnoliopsida,Platanaceae,Platanus,Plantae,Platanus occidentalis,Proteales,Magnoliophyta,Plantae
2,4736,train_val2018/Plantae/7477/c0843babf2fe1b24b6c...,7477,Magnoliopsida,Platanaceae,Platanus,Plantae,Platanus occidentalis,Proteales,Magnoliophyta,Plantae
3,10022,train_val2018/Plantae/7477/1645a80d3bb42125d3f...,7477,Magnoliopsida,Platanaceae,Platanus,Plantae,Platanus occidentalis,Proteales,Magnoliophyta,Plantae
4,10296,train_val2018/Plantae/7477/5b96d95868a7e97b97e...,7477,Magnoliopsida,Platanaceae,Platanus,Plantae,Platanus occidentalis,Proteales,Magnoliophyta,Plantae


In [29]:
len(val_df['kingdom'].unique())

2

In [51]:
# https://www.kaggle.com/hsinwenchang/keras-mobilenet-data-augmentation-visualize
numTrainingImages = len(train_df['image_id'])
numValidImages = len(val_df['image_id'])
print(numTrainingImages)




125664


In [31]:
val_df.head()

,image_id,file_name,category_id,class,family,genus,kingdom,name,order,phylum,supercategory
0,437514,train_val2018/Plantae/7291/9f8ecb39e5f767e39e9...,7291,Magnoliopsida,Euphorbiaceae,Euphorbia,Plantae,Euphorbia ophthalmica,Malpighiales,Magnoliophyta,Plantae
1,445585,train_val2018/Plantae/7291/a283eed3c7fcfd2b2f8...,7291,Magnoliopsida,Euphorbiaceae,Euphorbia,Plantae,Euphorbia ophthalmica,Malpighiales,Magnoliophyta,Plantae
2,460454,train_val2018/Plantae/7291/05bc377ee8066c43ac1...,7291,Magnoliopsida,Euphorbiaceae,Euphorbia,Plantae,Euphorbia ophthalmica,Malpighiales,Magnoliophyta,Plantae
6,437516,train_val2018/Plantae/7228/cfb9ca114758b134ae2...,7228,Magnoliopsida,Verbenaceae,Glandularia,Plantae,Glandularia pumila,Lamiales,Magnoliophyta,Plantae
7,438162,train_val2018/Plantae/7228/7f8fe3242e3bbe6e357...,7228,Magnoliopsida,Verbenaceae,Glandularia,Plantae,Glandularia pumila,Lamiales,Magnoliophyta,Plantae


In [32]:
# UNCOMMENT IF NEED TO GENERATE ANOTHER COPY OF SPECIES LIST

# Generate csv file containing info about all species 
# all_species_df = train_df.copy(deep=True)
# all_species_df = all_species_df.drop(columns=['image_id', 'file_name'])

# Remove duplicates
# keep = 'first' means the first entry of a set in duplicates remains in the DF
# all_species_df= all_species_df.drop_duplicates(keep='first')

# https://datatofish.com/export-dataframe-to-csv/

# all_species_df.to_csv('../nature-hunt/deep_learning_model_work/2018_species_list.csv')

# all_species_df.head()

In [33]:
# def retrieve_and_trim_17(path):
#     # Load Data from JSON
#     with open(path) as data_file:
#         anns = json.load(data_file)
#     # Read in Image + Category Pairs
#     anns_df = pd.DataFrame(anns['annotations'])[['image_id', 'category_id']]
    
#     # Read in Image + File_Name Pairs
#     img_df = pd.DataFrame(anns['images'])[['id', 'file_name']].rename(columns={'id':'image_id'})

#     # Merge 
#     df_file_cat_id_only = pd.merge(img_df, anns_df, on='image_id')
    
#     # Read in categories
#     df_file_cat_data = pd.DataFrame(anns['categories']).rename(columns={'id':'category_id'})
    
#     # Merge category data 
#     df_file_all = pd.merge(df_file_cat_data, df_file_cat_id_only, on='category_id')
    
#     # Retain only the records with the category as Plantae or Fungi
#     #df_file_w_kingdom = df_file_w_kingdom[df_file_w_kingdom['kingdom'].isin(['Plantae', 'Fungi'])]

#     return df_file_all[df_file_all['supercategory'].isin(['Plantae', 'Fungi'])]

In [34]:
# # Read in 2017 data

# path_to_train_data_17 = '../Downloads/train_val2017/train2017.json'
# path_to_val_data_17 = '../Downloads/train_val2017/val2017.json'

# # Retrieve and parse data
# train_df_17 = retrieve_and_trim_17(path_to_train_data_17)
# val_df_17 = retrieve_and_trim_17(path_to_val_data_17)

# train_df_17.head()


In [35]:
# len(val_df_17['category_id'].unique())

In [36]:
# # Gauge overlap between 2017 & 2018 data
# # 2017 = 2222 plant and fungi species total
# names_2017 = val_df_17['name'].drop_duplicates(keep='first').to_frame()

# # 2018 = 3238 plant and fungi species
# names_2018 = val_df['name'].drop_duplicates(keep='first').to_frame()

# merged_names = pd.merge(names_2017, names_2018, on='name')
# len(merged_names.index)


## Decided not to use any of the 2017 data

1. 2017 data doesn't include other informational data - genus, class, etc. I expect to use genus classification
as a "valid find" too for the scavenger hunt app, so that is important to have on hand.

2. There were 807 overlapping species between 2017 and 2018. I didn't want to include the 2017 
supplementary images to train on, as that would lead to a data imbalance - the species found in both 2017 and 2018
would have more data. Suggested to be problematic here: https://datascience.stackexchange.com/questions/38796/unbalanced-training-data-for-different-classes

# CHANGE NB_EPOCHS 

In [42]:
# Number of classes 
nb_classes = len(train_df['category_id'].unique())
print(nb_classes)

# Batch Size 
# https://stats.stackexchange.com/questions/164876/tradeoff-batch-size-vs-number-of-iterations-to-train-a-neural-network
# Tradeoff for batch size: larger batch size trains faster but uses more memory and may be less accurate

batch_size = 128
img_size = 128

# Going to run a one epoch model training to estimate how long each takes

nb_epochs = 1


3238


Consider using ModelCheckpoint or EarlyStopping

https://keras.io/callbacks/

 Questions to answer:
1. Number of Epochs?
2. Steps per epoch?
3. Validation steps?
4. Accuracy / Val Accuracy vs Loss / Val Loss

### In the cell below, use ImageDataGenerator to create a transformation that rescales the images by 255 

here I applied Data Augmentation as following:
random 45 degree rotation
random zoom of up to 50%
random horizontal flip
width shift of 0.15
height shfit of 0.15

# CHANGE DIRECTORY PARAMETER

In [38]:
%%time
# Training Data
train_datagenerator = ImageDataGenerator(rescale=1./255, rotation_range=45, 
                                         width_shift_range=.15,
                                        height_shift_range=.15,
                                        horizontal_flip=True,
                                        zoom_range=0.5)

train_generator = train_datagenerator.flow_from_dataframe(dataframe=train_df,
                                                          directory="../input/train_val2018",
                                                          x_col="file_name",
                                                          y_col="category_id",
                                                          batch_size=batch_size,
                                                          shuffle=True, #shuffles order of batches at beginning of each epoch
                                                          # use sparse not categorical because labels are integers not one-hot encoded : https://datascience.stackexchange.com/questions/39132/keras-dimension-error-with-sparse-categorical-crossentropy
                                                          class_mode="sparse", #https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy
                                                          target_size = (img_size, img_size))
                                                                                                          
                                                          
                                                          

Found 0 validated image filenames belonging to 0 classes.
CPU times: user 786 ms, sys: 137 ms, total: 923 ms
Wall time: 930 ms


/Users/kelson/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 125664 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


# CHANGE DIRECTORY PARAMETER

In [41]:
%%time
# Validation Data
test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=test_datagen.flow_from_dataframe(dataframe=val_df,
                                                directory="../input/train_val2018",
                                                 x_col="file_name",
                                                 y_xol="category_id",
                                                 batch_size=batch_size,
                                                 class_mode="sparse",
                                                 target_size=(img_size , img_size))
                                                
    

Found 0 validated image filenames belonging to 0 classes.
CPU times: user 77.5 ms, sys: 14.1 ms, total: 91.7 ms
Wall time: 89.6 ms


/Users/kelson/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 9714 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


## MobileNet Model

Initializes weights randomly, so does full training. An alternative Kaggle kernel uses ImageNet weights, freezes the model, adds a few on top and just trains those - example of FineTuning / Transfer Learning. 

I want to have more customization for better (hopefully) performance
Kernel here: https://www.kaggle.com/ateplyuk/inat2019-starter-keras-efficientnet

In [44]:
model = MobileNet(input_shape=(img_size, img_size,3), 
                  alpha=1., # alpha is "Width Multiplier"
                 weights=None, # use None along with custom classes parameter for random initialization of weights
                 classes=nb_classes)


# Link on overview of optimizers: http://ruder.io/optimizing-gradient-descent/index.html#rmsprop
# Seems like Adam and rmsprop are relatively similar 
# will use RMSprop because that's what is used in both Kaggle Kernels with high performance on similar data
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6), 
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

## Train Model

In [53]:
# Use callbacks to save the best model
# Further parameter explanations: https://keras.io/callbacks/

model_save_name = "MobileNet_1.h5"
checkpoint = ModelCheckpoint(model_save_name, # save destination
                             monitor='val_loss', # loss on validation data
                            verbose=1, #verbosity
                            save_best_only=True,
                            save_weights_only=False, # save whole model
                            mode='auto',
                            period=1) # Interval (number of epochs) between checkpoints



In [52]:
%%time 

# Specify number of steps = size of training data / batch size
trainingEpochSteps = numTrainingImages / batch_size
valEpochSteps = numValidImages / batch_size

history = model.fit_generator(generator = train_generator,
                             steps_per_epoch = trainingEpochSteps,  
                             validation_data = valid_generator,
                             validation_steps = valEpochSteps, 
                             epochs = nb_epochs,
                             verbose=0)


InvalidArgumentError: input and filter must have the same depth: 3 vs 32
	 [[{{node conv_dw_1_1/depthwise}}]]

In [ ]:
# Convert to TensorFlow Lite
# https://www.tensorflow.org/lite/convert/python_api

converter = tf.lite.TFLiteConverter.from_keras_model_file(model_save_name)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)